Open and reshape data

In [1]:
import pandas as pd
import numpy as np
import pickle
import ipdb
from os import listdir
import copy



In [2]:
def learners_data(where,files,name):
    Learners=[];
    for f in files:
        if f[-11:-4]=='learner':
            df = pd.read_csv(where+f, delimiter=',');
            tuples = [tuple(x) for x in df.values]
            dicts = df.to_dict().values()
            subject={};
            if name=='Id':
                subject['Id']=int(tuples[0][6][0:tuples[0][6].index('-')]);
                subject['age']=int(tuples[0][6][tuples[0][6].index('-')+1:]); 
                subject['gender']=f[f.index('_')-1:f.index('_')];
            else:
                subject['age']=int(tuples[0][6][0:tuples[0][6].index('-')]);
                subject['Id']=int(tuples[0][6][tuples[0][6].index('-')+1:]);
                if subject['Id'] in Id:
                    subject['gender']=genders[Id.index(subject['Id'])][0];    
                else:
                    subject['gender']=[];
                    print(str(subject['Id'])+' no encontrado en la planilla')
            subject['type']='learner';
            subject['start_time']=tuples[0][5];                           
            if tuples[10][10]==0:
                subject['understood']=True;
            else:
                subject['understood']=False;
            subject['trials']=[];    
            n_errors=0;
            for row in tuples:
                if row[1]=='learner' and row[11]==False:
                    circles=row[16];
                    crosses=row[17];
                    circles_list=[];
                    crosses_list=[];
                    if not(pd.isnull(circles)):           
                        circles=circles.split(",");            
                        for element in circles:                
                            if element[0]=='[':
                                element=element[1:];
                            if element[-1]==']':
                                element=element[0:-1];
                            circles_list.append(float(element));
                        circles_list = list(zip(*[iter(circles_list)] * 2))

                    if not(pd.isnull(crosses)):    
                        crosses=crosses.split(",");            
                        for element in crosses:
                            if element[0]=='[':
                                element=element[1:];
                            if element[-1]==']':
                                element=element[0:-1];
                            crosses_list.append(float(element));
                        crosses_list = list(zip(*[iter(crosses_list)] * 2))
                    if not(np.any([x<0 for x in row[12:16]])):
                        subject['trials'].append({'Circles':circles_list,'Crosses':crosses_list,'rt':row[7],'Box':row[12:16],'Consistent':row[18]})    
                        if not(row[18]): # not consistent
                            n_errors=n_errors+1;
            subject['inconsistent_trials']=n_errors/len(subject['trials']);    
            Learners.append(subject);
    return Learners;

###########################################################################################################    
###########################################################################################################
###########################################################################################################

def teachers_data(where,files,name):
    # where: location of files
    # files: list of files
    # name: format of file names: if name==Id the format is Id-age_*, elif name==age the format is age-Id_*
    Teachers=[];
    for f in files:  
        if f[-11:-4]=='teacher':
            df = pd.read_csv(where+f, delimiter=',')
            tuples = [tuple(x) for x in df.values]    
            subject={};    
            if name=='Id':
                subject['Id']=int(tuples[0][6][0:tuples[0][6].index('-')]);
                subject['age']=int(tuples[0][6][tuples[0][6].index('-')+1:]);                
                subject['gender']=f[f.index('_')-1:f.index('_')];
            else:            
                subject['age']=int(tuples[0][6][0:tuples[0][6].index('-')]);
                subject['Id']=int(tuples[0][6][tuples[0][6].index('-')+1:]);                 
                if subject['Id'] in Id:
                    subject['gender']=genders[Id.index(subject['Id'])][0];    
                else:
                    subject['gender']=[];
                    print(str(subject['Id'])+' no encontrado en la planilla')
                
            subject['type']='teacher';
            subject['start_time']=tuples[0][5];                                  
            if tuples[10][10]==0:
                subject['understood']=True;
            else:
                subject['understood']=False;
            subject['trials']=[];
            for row in tuples:
                if row[1]=='teacher' and row[11]==False:        
                    circles=row[16];
                    crosses=row[17];
                    circles_list=[];
                    crosses_list=[];
                    if not(pd.isnull(circles)):           
                        circles=circles.split(",");            
                        for element in circles:                
                            if element[0]=='[':
                                element=element[1:];
                            if element[-1]==']':
                                element=element[0:-1];
                            circles_list.append(float(element));
                        circles_list = list(zip(*[iter(circles_list)] * 2))

                    if not(pd.isnull(crosses)):    
                        crosses=crosses.split(",");            
                        for element in crosses:
                            if element[0]=='[':
                                element=element[1:];
                            if element[-1]==']':
                                element=element[0:-1];
                            crosses_list.append(float(element));
                        crosses_list = list(zip(*[iter(crosses_list)] * 2))                                    
                    subject['trials'].append({'Circles':circles_list,'Crosses':crosses_list,'rt':row[7],'Box':row[12:16]})        
            Teachers.append(subject)
    return Teachers;            

     

In [6]:
xls = pd.ExcelFile("/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Julio_2018/C3-Julio-Final.xlsx")
Hoja1=xls.parse(0);
subjects=Hoja1['ID ( Edad + Nº)'];
genders=Hoja1['Género'];

Id=[];
Age=[];
for s in subjects:
    age_id=s.split();    
    Id.append(int(age_id[1]));
    Age.append(int(age_id[0]));
    
'''

where1='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Junio_2018/Learner/';
where2='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Junio_2018/Teacher/';
where3='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Julio_2018/Learner/';
where4='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Julio_2018/Teacher/';


L1=learners_data(where1,listdir(where1),'Id');
T1=teachers_data(where2,listdir(where2),'Id');
L2=learners_data(where3,listdir(where3),'age');
T2=teachers_data(where4,listdir(where4),'age');

for l in L2:
    L1.append(l)
    
for t in T2:
    T1.append(t)
'''

"\n\nwhere1='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Junio_2018/Learner/';\nwhere2='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Junio_2018/Teacher/';\nwhere3='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Julio_2018/Learner/';\nwhere4='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Julio_2018/Teacher/';\n\n\nL1=learners_data(where1,listdir(where1),'Id');\nT1=teachers_data(where2,listdir(where2),'Id');\nL2=learners_data(where3,listdir(where3),'age');\nT2=teachers_data(where4,listdir(where4),'age');\n\nfor l in L2:\n    L1.append(l)\n    \nfor t in T2:\n    T1.append(t)\n"

In [7]:
genders

0      Femenino 
1      Masculino
2      Femenino 
3      Femenino 
4      Femenino 
5      Masculino
6      Masculino
7      Femenino 
8      Masculino
9      Femenino 
10     Femenino 
11     Masculino
12     Masculino
13     Femenino 
14     Masculino
15     Femenino 
16     Femenino 
17     Femenino 
18     Femenino 
19     Femenino 
20     Femenino 
21     Masculino
22     Femenino 
23     Masculino
24     Femenino 
25     Masculino
26     Femenino 
27     Femenino 
28     Femenino 
29     Masculino
         ...    
114    Femenino 
115    Masculino
116    Masculino
117    Masculino
118    Femenino 
119    Masculino
120    Masculino
121    Masculino
122    Masculino
123    Femenino 
124    Femenino 
125    Masculino
126    Masculino
127    Masculino
128    Masculino
129    Masculino
130    Femenino 
131    Femenino 
132    Femenino 
133    Masculino
134    Masculino
135    Femenino 
136    Masculino
137    Femenino 
138    Femenino 
139    Masculino
140    Femenino 
141    Femenin

##  Save structured data

In [ ]:
afile = open(r'/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Teachers.pkl', 'wb')
pickle.dump(T1, afile)
afile.close()

afile = open(r'/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Individual/Learners.pkl', 'wb')
pickle.dump(L1, afile)
afile.close()

new_Learners=copy.deepcopy(Learners);
new_Teachers=copy.deepcopy(Teachers);

#reload object from file
file2 = open(r'Teachers.pkl', 'rb')
Teachers = pickle.load(file2)
file2.close()

#reload object from file
file2 = open(r'Learners.pkl', 'rb')
Learners = pickle.load(file2)
file2.close()

for t in new_Teachers:
    Teachers.append(t);

for l in new_Learners:
    Learners.append(l)



#[print(item) for item in Learners if item['Id']==13]

In [38]:
# DECEMBER DATA

where3='/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Diciembre_2018/Learners/';
where4='/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Diciembre_2018/Teachers/';

Id=[]
Age=[]
L1=learners_data(where3,listdir(where3),'age');
T1=teachers_data(where4,listdir(where4),'age');

id_gender=[(5,'F'),(3,'M'),(12,'F'),(14,'F'),(16,'M'),(26,'F'),(31,'M'),(37,'M'),(28,'F'),(50,'F'),(51,'F'),(66,'M'),(68,'M'),(69,'M'),(11,'F')\
           ,(25,'M'),(29,'F'),(28,'F'),(49,'M'),(55,'M'),(67,'M'),(2,'M'),(17,'F'),(24,'F'),(35,'M'),(39,'M'),(56,'F'),(62,'F'),(73,'F')]

for subject in L1:    
    s_id=subject['Id']    
    for item in id_gender:        
        if item[0]==s_id:            
            subject['gender']=item[1]
            break

            
for subject in T1:    
    s_id=subject['Id']    
    for item in id_gender:        
        if item[0]==s_id:            
            subject['gender']=item[1]
            break

            


31 no encontrado en la planilla
69 no encontrado en la planilla
66 no encontrado en la planilla
49 no encontrado en la planilla
16 no encontrado en la planilla
25 no encontrado en la planilla
55 no encontrado en la planilla
3 no encontrado en la planilla
12 no encontrado en la planilla
14 no encontrado en la planilla
73 no encontrado en la planilla
56 no encontrado en la planilla
51 no encontrado en la planilla
72 no encontrado en la planilla
26 no encontrado en la planilla
29 no encontrado en la planilla
67 no encontrado en la planilla
28 no encontrado en la planilla
11 no encontrado en la planilla
68 no encontrado en la planilla
50 no encontrado en la planilla
24 no encontrado en la planilla
35 no encontrado en la planilla
17 no encontrado en la planilla


In [40]:
# LOAD TO PREVIOUS DATA


file2 = open(r'/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Teachers.pkl', 'rb')
Teachers = pickle.load(file2)
file2.close()

file2 = open(r'/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Learners.pkl', 'rb')
Learners = pickle.load(file2)
file2.close()


In [41]:
# APPEND TO PREVIOUDS DATA
for t in T1:
    Teachers.append(t)
    
for l in L1:
    Learners.append(l)



In [44]:
# SAVE

afile = open(r'/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Teachers.pkl', 'wb')
pickle.dump(Teachers, afile)
afile.close()

afile = open(r'/home/ahaimovici/Documents/Teaching/Rectangle_Game/C3/Data/Individual/Learners.pkl', 'wb')
pickle.dump(Learners, afile)
afile.close()